<a href="https://colab.research.google.com/github/AlexKressner/KI_Logistik_Python/blob/main/G2_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Übersicht
1. [Problem/ Business Understanding](#problem)
1. [Data Understanding & Data Exploration](#data)
  1. [Erster Überblick](#overview)
  1. [Target](#target)
  1. [Features](#feature)

# Übungsaufgaben
- [Data Exploration - Weindaten](#wine_eda)

## 1 Problem/ Business Understanding <a class="anchor" id="problem"></a>
Sie kennen bereits die Daten zu den Fussballweltmeisterschaften von 1930 bis 2014. In diesen ist jedes einzelne Weltmeisterschaftsspiel dokumentiert - von der Zuschauerzahl bis zum Endergebnis. 

Wir werden nun versuchen, die Anzahl der in einem Spiel geschossenen Tore nach Abschluss der ersten Halbzeit vorherzusagen. Durch die Nutzung von maschinellen Lernverfahren versuchen wir aus den zur Halbzeit vorliegenden Daten Muster zu erkennen, die eine zuverlässige Vorhersage der geschossenen Tore erlaubt. 

Zunächst überlegen wir uns, welche Daten (Features) einen Einfluss auf die Anzahl der geschossenen Tore haben. Mit Sicherheit ist in diesem Zusammenhang die Anzahl der zur Halbzeit geschossenen Tore eine wichtige Information. Zusätzlich kann man sich aber auch die folgenden Fragen stellen:
- Hat die Spielpaarung einen Einfluss auf die Anzahl der Tore?
- Hat die Anzahl der Zuschauer einen Einfluss auf die Anzahl der geschossenen Tore?
- Fallen mehr Tore in einem Vorrundenspiel im Vergleich zu einem Halbfinale?
- ...

Sicherlich finden wir auf jede einzelne Frage auch ohne maschinelle Lernverfahren durch eine tiefergehende Datenanalse erste Antworten. Die Bewertung der Einflüsse in ihrer Kombination ist aber schon äußerst schwierig.

Maschinelle Lernverfahren helfen hier enorm. Wir müssen lediglich die Daten in geeigneter Weise aufbereiten, einen passenden Algorithmus auf diese anwenden und erhalten anschließend Prognosen zur interessierten Zielgröße oder Aussagen über die Bedeutung einzelner Features.

**Frage:** Welchem Teilgebiet des maschinellen Lernens ist die oben beschriebene Problemstellung zuzuordnen? Supervised oder Unsupervised Learning?

**Frage:** Handelt es sich um ein Regressions- oder Klassifikationsproblem?

## 2 Data Understanding & Exploration <a class="anchor" id="data"></a>
In einem ersten Schritt ist es wichtig, dass Sie sich einen Überblick zu den vorhandenen Daten verschaffen, d.h.:
- Wie groß ist der Datensatz (Zeilen/Spalten)
- Was bedeuten die einzelnen Werte in den Spalten (Features)?
- Wie sind die Werte der einzelnen Features verteilt?
- Gibt es fehlende Werte, die Sie bereinigen müssen?
- Wie sin die Zusammenhänge zwischen den Features?
- ...

Besonders hilfreich sind an dieser Stelle Visualisierung, die Ihnen einen Überblick zu den Daten geben und bereits etwaige Zusammenhänge zwischen Features und/oder Target aufzeigen!

In [ ]:
import pandas as pd # zur Datenanalyse
import matplotlib.pyplot as plt # zur Datenvisualisierug
import numpy as np

In [ ]:
# Google-Drive einbinden
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Daten laden
data = pd.read_csv("/content/drive/MyDrive/WorldCupMatches.txt")
data = data.astype({"Datetime": "M", "RoundID": "O", "MatchID": "O"}, errors='raise') 
data.dropna(inplace=True)
data.replace(' ', np.nan, inplace=True)
data.head()

### 2.1 Erster Überblick zum Datensatz <a class="anchor" id="overview"></a>

In [ ]:
# Größe des Datensatzes
data.shape

In [ ]:
# Fehlende Werte
data.isna().any()

### 2.2 Target <a class="anchor" id="target"></a>
Bei dem Target handelt es sich um die Zielgröße, welche wir prognostizieren wollen. In diesem Fall die Anzahl der geschossenen Tore in der regulären Spielzeit plus etwaiger Verlängerung!

In [ ]:
# Berechnung neue Spalte
data["Total Goals"] = data["Home Team Goals"] + data["Away Team Goals"]

In [ ]:
data["Total Goals"].plot(
    kind="hist", 
    title= "Histogramm Total Goals",
    figsize=(8,4),
    color="c",
    edgecolor='k',
    density=True,
    alpha=0.8
    )
plt.plot()

**Frage:** Warum ist es unabdingbar die folgenden Spalten zu löschen?

In [ ]:
data.drop(columns=["Home Team Goals", "Away Team Goals"], inplace=True)

### 2.3 Features <a class="feature" id="overview"></a>
Bei den Features handelt es sich um die Variablen, die zur Vorhersage des Targets verwendet werden - in diesem Fall die Anzahl der geschossenen Tore. 

Zum einen müssen Sie zunächst überlegen, welche der ursprünglichen Features einen Beitrag zur Erklärung des Target leisten können und ob diese noch anzupassen sind. 

Zum anderen leitet man regelmäßig aus den ursprünglichen Features weitere Features ab - in der Annahme, dass diese einen Beitrag zur Vorhersage des Target leisten.

**Relevante Features**

In [ ]:
irrelevant = []

In [ ]:
data.drop(columns=irrelevant, inplace=True)
data.head()

**Features zu den zur Halbzeit geschossenen Toren**

In [ ]:
data["Half-time Goals"] = data["Half-time Home Goals"] + data["Half-time Away Goals"]
data["Half-time Goals Difference"] = abs(data["Half-time Home Goals"] - data["Half-time Away Goals"])

In [ ]:
data["Half-time Goals Difference"].unique()

In [ ]:
data["Half-time Goals Difference"].value_counts(normalize=True)

In [ ]:
#data.drop(columns=["Half-time Home Goals", "Half-time Away Goals"], inplace=True)

**Features zur Spielzeit**

**Frage:** Ist die Verwendung der Spalte `Datetime` mit den aktuellen Werten sinnvoll für die Prognose des Targets? Welche Informationen lassen sich aus dem genauen Spieldatum inkl. Uhrzeit ableiten, die ggf. für die Vorhersage des Targets relevant sind?

**`Datetime`-Objekte in Python**

Die Werte in der Spalte `Datetime` sind vom Typ `Datetime` und stellen ein besonderes Objekt zur Codierung von Datum und Uhrzeit dar. Für `Datetime`-Objekte stehen wiederum spezielle Methoden zur Verfügung, die wir nachfolgend anwenden werden. Eine Übersicht zu den Methoden finden Sie z.B. [hier](https://towardsdatascience.com/working-with-datetime-in-pandas-dataframe-663f7af6c587).

In [ ]:
# Ableitung der Spielzeit (Stunde)
data["Datetime"].dt.hour.value_counts()

In [ ]:
data["Hour"] = data["Datetime"].dt.hour

In [ ]:
# Ableitung des Spieltags (Wochentag)
data["Datetime"].dt.day_name().value_counts()

In [ ]:
data["Weekday"] = data["Datetime"].dt.day_name()

In [ ]:
data.drop(columns="Datetime", inplace=True)

**Feature zur Spielrunde**

In [ ]:
# Spielrunde
data.Stage.unique()

Die Verarbeitung von Strings ist ein eigenes Kapitel für sich, welches wir nicht in der Vorlesung vertiefen und mit dem Sie sich auch in der Fallstudie nicht detailliert auseinandersetzen müssen. Beispielhaft sei hier die Aufbereitung der Werte in der Spalte `Stage` behandelt.

In [ ]:
data.Stage.replace(r'Group(.*)', 'Group', regex=True, inplace=True)

In [ ]:
data.Stage.replace('First round', 'Group', inplace=True)

In [ ]:
data.Stage.replace('Preliminary round', 'Group', inplace=True)

In [ ]:
data.Stage.replace(r'.+third.+', 'Third place', regex=True, inplace=True)

In [ ]:
data.Stage.value_counts()

Nach den vorausgegangenen Datenmanipulationen erhalten wir nun den folgenden Dataframe (Target + Features), mit dem wir bei der Anwendung maschineller Lernverfahren weiterarbeiten werden.

In [ ]:
data.head()

### 2.4 Korrelationen <a class="feature" id="corr"></a>
Sie erhalten einen ersten sehr guten Eindruck von Zusammenhängen zwischen Features und/ oder Target, wenn Sie Korrelationskoeffizienten berechnen. Mithilfe von `pandas` gelingt dies mühelos.


In [ ]:
correlation_matrix = data.corr()

**Korrelationskoeffizient**

Mit der `pandas`-Funktion `.corr()` berechnen Sie den [Korrelationskoeffizienten nach Pearson](https://de.wikipedia.org/wiki/Korrelationskoeffizient). Dieser misst den "linearen" Zusammenhang zwischen zwei stetigen Merkmalen. Der Koeffizient reicht von 1 bis -1. Für den Fall, dass er nahe 1 liegt, bedeutet dies, dass eine starke positive Korrelation besteht, d.h. "wenn Werte des Merkmals `x` steigen, steigen auch solche des Merkmals `y`"! Der umgekehrte Fall gilt für Koeffizienten, die nahe -1 liegen. Bitte behalten Sie in Erinnerung, dass der Korrelationskoeffizient nach Pearson nur lineare Zusammenhänge erfasst. Regelmäßig finden Sie aber auch nicht-lineare Korrelationen, die dann nicht erfasst werden ([Beispiel](https://de.wikipedia.org/wiki/Korrelationskoeffizient#/media/Datei:Correlation_examples.png)).

In [ ]:
correlation_matrix["Total Goals"].sort_values(ascending=False)

Neben Korrelationskoeffizienten können Sie Zusammenhänge zwischen Merkmalen auch immer visuell analysieren. Dazu bieten sich insbesondere Streudiagramme an (englisch: Scatter Plot).

In [ ]:
data.plot(kind="scatter", x="Half-time Goals", y="Total Goals", alpha=0.1)
plt.show()

In [ ]:
pd.plotting.scatter_matrix(data[["Total Goals", "Half-time Goals", "Half-time Goals Difference"]], figsize=(12,8))
plt.show()

**Kreuztabellen und Heatmaps**

In [ ]:
data.groupby("Weekday")["Total Goals"].mean().sort_values(ascending=False)

In [ ]:
pd.crosstab(index=data["Weekday"], columns=data["Total Goals"])

Das Package `seaborn` ist eine Erweiterung der Standardfunktionalitäten von `matplotlib`, welches wir bisher verwendet haben. Mit `seaborn` lassen sich optisch sehr ansprechende Visualisierung auf recht einfache Art und Weise erstellen. Immer wieder gerne genutzt wird die Visualisierungsform einer **Heatmap**, die sehr intuituve den Zusammenhang zwischen Variablen wiedergibt. Detaillierte Hinweise zur Erstellung von Heatmaps finden Sie [hier](https://seaborn.pydata.org/generated/seaborn.heatmap.html).

In [ ]:
import seaborn as sns

In [ ]:
crosstab = pd.crosstab(index=data["Weekday"], columns=data["Total Goals"], normalize=True)

In [ ]:
ax = sns.heatmap(crosstab, linewidths=.5, cmap="bwr")


Über das Argument `cmap` steuern Sie die farbliche Darstellung der Heatmap. Sämtliche Farbpaletten finden Sie [hier](https://matplotlib.org/stable/tutorials/colors/colormaps.html).

**Frage:** Wie erstellen Sie eine Heatmap, die den Zusammenhang zwischen Anzahl geschossener Tore (`Total Goals`) und der Spielrunde (`Stage`) darstellt?

### Übung EDA Wine Data Set <a class="anchor" id="wine_eda"></a>
**Problem Statement**

Sie haben sich dazu entschlossen einen e-Commerce Shop für Weinspezialitäten zu eröffnen. Ein wesentlicher Erfolgsfaktor ihres Geschäftsmodells besteht darin, Weine günstig einzukaufen und anschließend zu einem deutlich höheren Preis zu verkaufen. Zu den Einkaufspreisen diverser Weine verschaffen Sie sich regelmäßig bei verschiedenen Herstellern einen Überblick. Um den potentiellen Verkaufspreis eines neuen Weins zu ermitteln, vergleichen Sie diesen mit Verkaufspreisen ähnlicher Weine und leiten ihren Angebotspreis ab. Dazu steht Ihnen der folgende Datensatz zur Verfügung

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/KI_LOG_mit_PYTHON/wine.txt", index_col=0)
df.head()

Das oben kurz skizzierte Vorgehen ist sehr aufwendig und Sie haben das Gefühl, dass dieses auch nicht ganz gut funktioniert. Aus diesem Grund beschließen Sie ein Machine Learning Modell zu verwenden, welches die Zusammenhänge zwischen den am Markt angebotetnen Preisen eines Weins und ausgewählten Eigenschaften (Herkunft, Punkte, etc.) lernt. Das Modell möchten Sie anschlißend zu Berechnung realsitischer Angebotspreise verwenden. Auf diese Weise können Sie ohne großen Aufwand besonders margenträchtige (Einkaufspreis beim Produzenten minus prognostizierter Verkaufspreis) Weine identifizieren.

**Data Exploration**

Nehmen Sie sich die vorherigen Ausführungen als Ausgangsbeispiel und führen Sie eine Datenexploration als Grundlage für die Anwendung eines Machine Learning Verfahrens durch. Untersuchen Sie beispielsweise die Größe des Datensatzes, den Wertebereich des Targets und der Features, Korrelationen zwischen den Variablen, etc. Nutzen Sie dazu auch geeignete Grafiken!